In [1]:
import pandas as pd
import numpy as np
import collections

In [9]:
df = pd.read_csv('datasets/dataset_games.csv')
t = df
del df['release_date']
del df['Unnamed: 0']

In [10]:
# url_data = pd.read_csv('datasets/url_images.csv', index_col=False)
# del url_data['Unnamed: 0']
# url_data

In [11]:
url_links = pd.read_csv("datasets/urls_to_games.csv", index_col=False)

In [12]:
df['description'] = df['description'].fillna('None')

In [13]:
df['user_score'] = np.where(df['user_score'] == 'tbd',df['meta_score']/10 ,df['user_score'])
df['user_score'] = df['user_score'].astype(float)
df.dtypes

name_game       object
meta_score       int64
user_score     float64
platform        object
description     object
dtype: object

In [14]:
w = df[df.duplicated(subset='name_game', keep='first')]

In [15]:
duplicates = list(sorted(set(w['name_game'])))

In [16]:
duplicates[1]

'.hack//Outbreak Part 3'

In [17]:
# test 
names =  [[] for i in range(len(duplicates))]
meta_score = [[] for i in range(len(duplicates))]
user_score = [[] for i in range(len(duplicates))]
platform = [[] for i in range(len(duplicates))]
description = [[] for i in range(len(duplicates))]


for pos, item in enumerate(duplicates):
    for r in zip(*t.to_dict("list").values()):
        if item == r[0]:
            if item not in names[pos]:
                names[pos].append(r[0])
                
            if r[3] not in platform[pos]:
                meta_score[pos].append(r[1])
                user_score[pos].append(r[2])
                platform[pos].append((r[3]))
                description[pos].append(r[4])

In [18]:
def checker(des):
    if des[0] == '[':
        for x in range(len(des)):
            if des[x] == ']':
                return x + 1

In [19]:

def info(name, meta_score, user_score, platform, description, index):
    avg_meta = round((sum(meta_score[index]))/len(meta_score[index]), 2)
    avg_user = round((sum(user_score[index]))/len(user_score[index]), 2)

    
    des = max(description[index], key=len)
    
    test = checker(des)
    if test != None:
        des = des[test:].lstrip()
    else:
        des = des.lstrip()
        
    return name[index], avg_meta, avg_user, platform[index], des

In [20]:
# numpy loop
name_game_ = np.array([])
meta_score_ = np.array([])
user_score_ = np.array([])
platform_ = []
description_ = np.array([])
counter = 0

for x in range(len(duplicates)):
    counter+=1
    data = info(duplicates, meta_score, user_score, platform, description, x)
    name_game_  = np.append(name_game_, data[0])
    meta_score_ = np.append(meta_score_, data[1])
    user_score_ = np.append(user_score_, data[2])
    platform_.append(data[3])
    description_ = np.append(description_, data[4])

df2 = pd.DataFrame({
    'name_game': name_game_,
    'meta_score': meta_score_,
    'user_score': user_score_,
    'platform': platform_,
    'description': description_,
})

In [21]:
df = df[~df['name_game'].isin(duplicates)].reset_index(drop=True)
df

,name_game,meta_score,user_score,platform,description
0,The Legend of Zelda: Ocarina of Time,99,9.1,Nintendo 64,"As a young boy, Link is tricked by Ganondorf, ..."
1,Super Mario Galaxy,97,9.1,Wii,[Metacritic's 2007 Wii Game of the Year] The u...
2,Super Mario Galaxy 2,97,9.1,Wii,"Super Mario Galaxy 2, the sequel to the galaxy..."
3,Metroid Prime,97,8.9,GameCube,Samus returns in a new mission to unravel the ...
4,Super Mario Odyssey,97,8.9,Switch,New Evolution of Mario Sandbox-Style Gameplay....
...,...,...,...,...,...
4254,SPOGS Racing,18,3.7,Wii,[WiiWare] Race along in distinctively unique ...
4255,Double Dragon II: Wander of the Dragons,17,1.5,Xbox 360,A remake the 80's Double Dragon II game. Team ...
4256,Vroom in the Night Sky,17,3.2,Switch,Vroom in the night sky is a magical bike actio...
4257,Yaris,17,4.4,Xbox 360,[Xbox Live Arcade] Hop into a Toyota Yaris an...


In [22]:
df = df.append(df2).reset_index(inplace=False, drop=True)

In [23]:
df['user_score'] = df['user_score'].apply(lambda x: round(x * 10,2))
#df.to_csv('final_dataset.csv', index=False)

In [24]:
def get_list_lower(x):
    if isinstance(x, list): # For tested
        return [str.lower(i.replace(" ", "-")) for i in x]
    else:
        return [x.lower().replace(' ', '-')]
            
    # Return empty list in case of missing/malformed data
    return []

In [25]:
df['platform'] = df['platform'].apply(get_list_lower)

In [29]:
df['url_link'] = url_links

In [30]:
df.head(10)

,name_game,meta_score,user_score,platform,description,url_link
0,The Legend of Zelda: Ocarina of Time,99.0,91.0,[nintendo-64],"As a young boy, Link is tricked by Ganondorf, ...",https://www.metacritic.com/game/nintendo-64/th...
1,Super Mario Galaxy,97.0,91.0,[wii],[Metacritic's 2007 Wii Game of the Year] The u...,https://www.metacritic.com/game/wii/super-mari...
2,Super Mario Galaxy 2,97.0,91.0,[wii],"Super Mario Galaxy 2, the sequel to the galaxy...",https://www.metacritic.com/game/wii/super-mari...
3,Metroid Prime,97.0,89.0,[gamecube],Samus returns in a new mission to unravel the ...,https://www.metacritic.com/game/gamecube/metro...
4,Super Mario Odyssey,97.0,89.0,[switch],New Evolution of Mario Sandbox-Style Gameplay....,https://www.metacritic.com/game/switch/super-m...
5,Halo: Combat Evolved,97.0,87.0,[xbox],"Enter the mysterious world of Halo, an alien p...",https://www.metacritic.com/game/xbox/halo-comb...
6,The House in Fata Morgana - Dreams of the Reve...,97.0,83.0,[switch],A gothic suspense tale set in a cursed mansion...,https://www.metacritic.com/game/switch/the-hou...
7,NFL 2K1,97.0,62.0,[dreamcast],"In the end, NFL 2K1 is a deeper, more refined ...",https://www.metacritic.com/game/dreamcast/nfl-2k1
8,Uncharted 2: Among Thieves,96.0,88.0,[playstation-3],Fortune hunter Nathan Drake returns in Unchart...,https://www.metacritic.com/game/playstation-3/...
9,Tekken 3,96.0,91.0,[playstation],"An ancient evil force has reawakened, attackin...",https://www.metacritic.com/game/playstation/te...


In [31]:
df.to_csv('datasets/final_dataset.csv', index=False)